In [8]:
import os
import cv2
import glob
import sys 
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import csv
from src import PROJECT_DIR, MODEL_DIR, EXTRA_DIR, DATA_DIR
from src.dataset import VDAODataset
from src.resnet import Resnet50
from src.utils import threshold, opening, closing, erosion, dilation
import pandas  as pd

import math
from functools import reduce    
from src.alignment_utils import apply_optical_flow
from skimage import transform as tf
import time
import PIL.Image as Image

from skimage.measure import ransac
from skimage.registration import optical_flow_tvl1
from skimage.transform import EuclideanTransform

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def calculate_SRMSE(img_1,
                    img_2,
                    disconsider_region=None,
                    transf_to_grayscale=True,
                    border_percentage_removal=None):

    diff = img_1 - img_2

    if isinstance(diff, torch.Tensor):
        diff = diff.numpy()
    total_pixels = diff.size

    if disconsider_region is not None:

        mask = (disconsider_region < 0.1).astype(np.uint8)
        total_pixels = mask.sum()

        diff = diff*mask
        rmse = math.sqrt((diff**2).sum() / total_pixels)

    rmse = math.sqrt(np.mean(np.square(diff)))
    if diff.dtype == np.uint8:
        abs_diff_img = abs(diff)
    else:
        abs_diff_img = (255 * abs(diff)).astype(np.uint8)
    return {'rmse': rmse, 'abs_diff_img': abs_diff_img}

def apply_optical_flow(ref_image,
                       moving_image,
                       sil_image,
                       bounding_box=None,
                       min_samples=10,
                       apply_ransac=True,
                       ransac_res_only=False,
                       transformations=None,
                       prefilter=False,
                       get_optical_vectors_ransac=False,
                       res_threshold=0.1,
                       border_percentage_removal=0.05):

    if isinstance(ref_image, torch.Tensor):
        ref_image = ref_image.numpy()
    if isinstance(moving_image, torch.Tensor):
        moving_image = moving_image.numpy()
    if isinstance(sil_image, torch.Tensor):
        sil_image = sil_image.numpy()
    if ref_image.dtype != np.uint8:
        ref_image = (255 * ref_image).astype(np.uint8)
    if moving_image.dtype != np.uint8:
        moving_image = (255 * moving_image).astype(np.uint8)
    if sil_image.dtype != np.uint8:
        sil_image = (255 * sil_image).astype(np.uint8)

    if transformations is not None:
        ref_image = transformations(Image.fromarray(ref_image))
        ref_image = (255 * ref_image.squeeze().permute(1, 2, 0).numpy())
        ref_image = ref_image.astype(np.uint8)
        moving_image = transformations(Image.fromarray(moving_image)).squeeze()
        moving_image = (255 * moving_image.permute(1, 2, 0).numpy())
        moving_image = moving_image.astype(np.uint8)

    if isinstance(ref_image, Image.Image):
        ref_image = np.array(ref_image)
    if isinstance(moving_image, Image.Image):
        moving_image = np.array(moving_image)

    # Optical flow is applied into grayscale images
    ref_image_gs = ref_image
    moving_image_gs = moving_image

    nr, nc = ref_image_gs.shape

    # deslocamento vertical (v) e horizontal (u) considera que a ref_frames
    # é a deslocada
    v, u = optical_flow_tvl1(moving_image_gs, ref_image_gs, tightness=0.1,
                             num_warp=20, tol=1e-7, prefilter=prefilter)

    row_coords, col_coords = np.meshgrid(np.arange(nr),
                                         np.arange(nc),
                                         indexing='ij')

    if apply_ransac:

        src = np.array((row_coords.flatten(), col_coords.flatten())).T
        dst = np.array(((row_coords + u).flatten(),
                        (col_coords + v).flatten())).T
        # Fita modelo eliminando outliers com ransac
        error = np.sqrt((src - dst)**2).sum(axis=1)
        t = np.sqrt(5.99 * error.std())
        # Aplica ransac para retirar outliers
        model_ransac, inliers = ransac(
            (dst, src),
            EuclideanTransform,
            # min_samples=int((1 - 0.2) * (nc * nr)),  # Testar outros valores
            min_samples=min_samples,
            residual_threshold=t*res_threshold,  # Testar outros valores
            max_trials=50)

        # Gera a imagem com vetores com a aplicação do ransac
        img_vectors_ransac = None

        # Gera imagem retificada em cima da referencia em grayscale
        ref_ret_ransac_gs = tf.warp(ref_image_gs, model_ransac.inverse)

        # Calcula RMS entre o frame alvo grayscale e o de referência
        # transformado e em grayscale e desconsiderando
        rms_ransac = calculate_SRMSE(
            moving_image_gs,
            ref_ret_ransac_gs,
            border_percentage_removal=border_percentage_removal)

        abs_diff_noborders = rms_ransac['abs_diff_img']
        rms_ransac_full_frame = rms_ransac['rmse']
        # Tentativa: Dividindo pelo std
        # _ref_image_no_border = disconsider_borders(ref_image_gs,
        # border_percentage_removal)
        # rms_ransac = calculate_SRMS(abs_diff_noborders) /
        # _ref_image_no_border.std()
        # Se foi passado algum bounding box, calcula o RMS
        # desconsiderando a area do bounding box

        repositioned_bb = None
        rms_ransac_outside_bb_only = rms_ransac['rmse']
        if bounding_box is not None:

            rms_ransac_outside_bb_only = calculate_SRMSE(
                moving_image_gs,
                ref_ret_ransac_gs,
                disconsider_region=sil_image[0],
                border_percentage_removal=border_percentage_removal)
            rms_ransac_outside_bb_only = rms_ransac_outside_bb_only['rmse']
    else:
        img_vectors_ransac, model_ransac, inliers, rms_ransac,
        rms_ransac_outside_bb_only, abs_diff_noborders, ref_ret_ransac_gs,
        repositioned_bb = None, None, None, None, None, None, None, None

    if ref_ret_ransac_gs is not None:
        ref_ret_ransac_gs = (255 * ref_ret_ransac_gs).astype(np.uint8)
    if ransac_res_only:
        return {
            'img_ret_ransac_grayscale': ref_ret_ransac_gs,
            'img_ret_ransac_abs_diff_noborders': abs_diff_noborders,
            'model_ransac': model_ransac,
            'rms_ransac': rms_ransac_full_frame,
            'rms_ransac_outside_bb_only': rms_ransac_outside_bb_only,
            'repositioned_bb': repositioned_bb,
            'inliers_ransac': inliers,
            'img_optical_vectors_ransac': img_vectors_ransac,
            'optical_flow_components': [v, u],
        }


In [6]:
def find_best_reference(ref_frame, tar_frame, sil_frame, geometric=False):


    if geometric:
        res = apply_optical_flow(ref_frame,
                                tar_frame,
                                sil_frame,
                                bounding_box=None,
                                apply_ransac=True,
                                ransac_res_only=True,
                                transformations=None)

        model_ransac = res['model_ransac']    

        # warping
        ref_frame_tr = tf.warp(ref_frame, model_ransac.inverse)
        ref_frame_tr = (255*ref_frame_tr).astype('uint8')

        # cropping
        percentage = 0.025
        h, w = ref_frame_tr.shape[0], ref_frame_tr.shape[1]
        h_remove = int(h * percentage)
        w_remove = int(w * percentage)

        ref_frame_cr = ref_frame_tr[h_remove:h - h_remove, w_remove:w - w_remove,:]
        tar_frame_cr = tar_frame[h_remove:h - h_remove, w_remove:w - w_remove,:]
    else:
        ref_frame_cr = ref_frame
        tar_frame_cr = tar_frame

    diff = (ref_frame_cr - tar_frame_cr)**2
    rmse = math.sqrt(np.sum(diff)/ (ref_frame_cr.shape[0]*ref_frame_cr.shape[1]))
    
    return rmse

In [14]:
window = 10
total_time = 0
for fold in range(1,10):
    print(fold)
    dataset = VDAODataset(fold = fold, split = 0, type = 'test', alignment = 'temporal', transform = False)
    align_df = dataset.align_df
    videos = align_df.test_file.unique()

    for vv in videos:
        dd = align_df[align_df.test_file==vv]
        vv_idx = dd.index

        tar_frames = [dataset.__getitem__(ii)[1] for ii in vv_idx]
        ref_frames = [dataset.__getitem__(ii)[0] for ii in vv_idx]
        sil_frames = [dataset.__getitem__(ii)[2] for ii in vv_idx]

        time_0 = time.time()
        for frm_idx in range(201):

            candidates = np.arange(max(0,frm_idx-window),min(201,frm_idx+window+1))
            results    = np.zeros(len(candidates))

            results = [find_best_reference(tar_frames[frm_idx], ref_frames[i], sil_frames[frm_idx], geometric=False) for i in candidates]
            best_candidate  = int(candidates[np.argmin(results)])
            ref_frame_idx = int(candidates[np.argmin(results)])
        time_1 = time.time()

        total_time += time_1-time_0
print(total_time)

1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
10

In [9]:
window = 10
total_time = 0
for fold in range(1,10):
    print(fold)
    dataset = VDAODataset(fold = fold, split = 0, type = 'test', alignment = 'temporal', transform = False)
    align_df = dataset.align_df
    videos = align_df.test_file.unique()

    for vv in videos:
        dd = align_df[align_df.test_file==vv]
        vv_idx = dd.index

        tar_frames = [dataset.__getitem__(ii)[1] for ii in vv_idx]
        ref_frames = [dataset.__getitem__(ii)[0] for ii in vv_idx]
        sil_frames = [dataset.__getitem__(ii)[2] for ii in vv_idx]

        time_0 = time.time()
        for frm_idx in range(201):

            candidates = np.arange(max(0,frm_idx-window),min(201,frm_idx+window+1))
            results    = np.zeros(len(candidates))

            results = [find_best_reference(tar_frames[frm_idx], ref_frames[i], sil_frames[frm_idx], geometric=True) for i in candidates]
            best_candidate  = int(candidates[np.argmin(results)])
            ref_frame_idx = int(candidates[np.argmin(results)])
        time_1 = time.time()

        total_time += time_1-time_0
print(total_time)

1


ValueError: too many values to unpack (expected 2)